##Inicialização das bibliotecas do Python, leitura do arquivo csv e cálculo das médias móveis

###Inicializando as bibliotecas do Python

***https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html***

In [ ]:
from datetime import datetime
from urllib import request

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import os
plt.style.use('fivethirtyeight')
%matplotlib inline


##Leitura dos arquivos CSV e preparação dos dataframes

###<font color = gray> Lista contendo os anos onde se tem o histórico de energia vertida turbinável

In [ ]:
ano = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

###<font color = gray> Criação de uma lista contendo todos os dataframes que recebe os dados da URL e o nome de todos os arquivos disponíveis em:
https://ons-dl-prod-opendata.s3.amazonaws.com/dataset/energia_vertida_turbinavel_ho/ENERGIA_VERTIDA_TURBINAVEL_{ano}.csv

In [ ]:
evts = []

for i in range(len(ano)):
    request.urlretrieve(f'https://ons-dl-prod-opendata.s3.amazonaws.com/dataset/energia_vertida_turbinavel_ho/ENERGIA_VERTIDA_TURBINAVEL_{ano[i]}.csv'
, f'ENERGIA_VERTIDA_TURBINAVEL_{ano[i]}.csv' )
    evts.append(pd.read_csv(f'ENERGIA_VERTIDA_TURBINAVEL_{ano[i]}.csv', sep=';',encoding = "utf-8", parse_dates=['din_instante']))

evt = pd.concat(evts)

In [ ]:
evt.head()

In [ ]:
evt.tail()

###<font color = gray> Criação de novas colunas no dataframe

In [ ]:
evt.insert(loc = 1, column = 'data', value = evt['din_instante'].dt.date)
evt.insert(loc = 1, column = 'ano', value = evt['din_instante'].dt.year)

In [ ]:
evt.head()

###<font color = gray> Somando os valores de energia vertida turbinável por agente, mês e ano

In [ ]:
df = evt.groupby(['id_subsistema', 'nom_agente', 'nom_reservatorio', 'cod_usina', 'ano', ])['val_energiavertidaturbinavel'].sum().reset_index()
df.sort_values(['id_subsistema', 'nom_agente', 'cod_usina', 'ano'], inplace = True)
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
df.head(50)

###<font color = gray> Segregando os dataframes por ano, e selecionando os 15 maiores valores de energia vertida turbinável de cada ano.


In [ ]:
evt_ano = []
for i in range(len(ano)):
    evt_ano.append(df[df['ano'] == ano[i]].sort_values(by = ['val_energiavertidaturbinavel'],  ascending=False).head(15))
evt_ano[8]

###<font color = gray> Somando os valores de energia vertida turbinável por mês e ano

In [ ]:
df1 = evt.groupby([ 'data', ])['val_energiavertidaturbinavel'].sum().reset_index()
df1.sort_values([ 'data'], inplace = True)

In [ ]:
df1.head(50)

# Plotagem das energias vertidas turbináveis dos dataframes

###<font color = gray> Evolução da Energia Vertida Turbinável do SIN

In [ ]:
def imprimir():

    plt.figure(figsize = (15, 5))
    plt.plot(df1['data'], df1['val_energiavertidaturbinavel'], label = 'EVT', linestyle=':', linewidth = 1, color = "red")
    plt.title(f'Evolução da Energia Vertida Turbinável')
    plt.xlabel('data')
    plt.ylabel('EVT')
    plt.legend(loc = 'upper left')
    plt.show()

In [ ]:
imprimir()

###<font color = gray> Ranking das usinas do SIn segundo a Energia Vertida Turbinável por ano

In [ ]:
def plot_evt(i):
    fig, ax = plt.subplots( figsize = (15, 5))
    bar_colors = ['indigo', 'darkviolet', 'darkmagenta', 'darkred', 'red', 'orangered', 'darkorange', 'orange', 'gold', 'yellow', 'honeydew', 'lawngreen', 'green', 'darkgreen', 'blue' ]
    valores = evt_ano[i]['val_energiavertidaturbinavel'].tolist()
    usinas = evt_ano[i]['nom_reservatorio'].tolist()
    ax.bar(usinas, valores, color=bar_colors)
    ax.set_ylabel('Energia Vertida Turbinável (MWh)')
    ax.xaxis.set_tick_params(rotation=75)
    plt.rcParams['xtick.labelsize'] = 10
    plt.rcParams['ytick.labelsize'] = 10
    ax.set_title(f'Valores de energia vertida turbinável para o ano de {ano[i]}')
    plt.show()


In [ ]:
for i in range(len(ano)):
    plot_evt(i)
    print('___________________________________________________________________________________________________________________________________________________________________________________')